# **Récupération et formatage des données**
---


## Introduction<a class="anchor" id="partie1"></a>

Ce notebook contient les codes nécessaires à la récupération et au formatage des données. Les données ont été récupéré sur le site de [l'OCDE](https://data-explorer.oecd.org/) et sur celui [d'Eurostat](https://ec.europa.eu/eurostat/fr/data/database). 

Nous récupérons les données en csv et les transformons en un unique csv contenant pour chacun des pays les valeurs trimestrielles (de Q1 en 1995 à Q3 de 2024 <span style="color:red ; font-size:20px;"> faire attention les working hours ne vont qu'au Q3 de 2024 </span>) de PIB, d'heure de travail, des taux d'intérêts à court et long terme et de l'indice à la consommation des prix. Pour l'instant il nous manque aussi les anticipations des taux et les rendements des marchés financiers. Le csv final de ce notebook servira de base de données par la suite. 

Les pays retenus sont ceux pour lesquels on a réussi à collecter toutes les données relatives aux variables d'intérêt: Autriche (Austria) Belgique (Belgium) Bulgarie (Bulgaria) Tchéquie (Czechia) Danemark (Denmark) Estonie (Estonia) Finlande (Finland) France (France) Allemagne (Germany) Grèce (Greece) Hongrie (Hungary) Islande (Iceland) Irlande (Ireland) Italie (Italy) Lettonie (Latvia) Lituanie (Lithuania) Luxembourg (Luxembourg) Pays-Bas (Netherlands) Norvège (Norway) Pologne (Poland) Portugal (Portugal) Roumanie (Romania) Slovaquie (Slovakia) Slovénie (Slovenia) Espagne (Spain) Suède (Sweden) Suisse (Switzerland) Royaume-Uni (United Kingdom).


## Sommaire

* [Introduction](#partie1)
* [Récupération des données](#partie2)
    * [Données de PIB](#partie21)
    * [Données d'heures de travail](#partie22)
    * [Données des taux d'intérêts](#partie23)
    * [Données d'indice à la consommation des prix](#partie24)
    * [Données de croissance potentielle](#partie25)
* [Création de la base de données](#partie3)


## Récupération des données <a class="anchor" id="partie2"></a>

In [108]:
%pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\enriq\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [109]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.metrics import mean_squared_error

Nous créons la liste qui contient les pays cités précédemment, à noter que la Slovaquie apparaît sous le nom Slovakia pour Eurostat et Slovak Republic pour l'OCDE, elle est donc sous deux noms dans notre liste de pays à sélectionner. 

In [110]:
# Liste comportant les pays choisis
countries_selected=[
    "Austria", "Belgium", "Bulgaria", "Czechia", "Czech Republic", "Denmark", "Estonia", 
    "Finland", "France", "Germany", "Greece", "Hungary", "Iceland", 
    "Ireland", "Italy", "Latvia", "Lithuania", "Luxembourg", "Netherlands", 
    "Norway", "Poland", "Portugal", "Romania", "Slovakia", "Slovak Republic",
    "Slovenia", "Spain", "Sweden", "Switzerland", "United Kingdom"
]

### Données de PIB <a class="anchor" id="partie21"></a>

Nous extrayons les données relatives au PIB depuis Eurostat


In [111]:
# Chargement des données 
df_GDP = pd.read_csv("Données_extraites/GDP_trimestriel_eurostat.csv", encoding='utf-8')

# Sélection des colonnes nécessaires avec les pays sélectionnés
df_GDP_selected = df_GDP[df_GDP['geo'].isin(countries_selected)][['geo', 'TIME_PERIOD', 'OBS_VALUE']]

# Modification du format pour avoir les périodes en index et les noms des pays en nom de colonnes
df_GDP = df_GDP_selected.pivot_table(index='TIME_PERIOD', columns='geo', values='OBS_VALUE')

# Transformation des colonnes en PIB_Nom_du_pays
df_GDP.columns = [f'PIB_{col}' for col in df_GDP.columns]
df_GDP.columns = [col.replace(" ", "_") for col in df_GDP.columns]



### Données de d'heure de travail <a class="anchor" id="partie22"></a>

Nous extrayons les données relatives aux heures de travail depuis Eurostat. Nous avons trois base de données pour les heures de travail. Une base annuelle de 1995 à 1997, et deux bases trimestrielles allant du Q1 de 1998 au Q4 de 2007 et du Q1 de 2008 au Q3 de 2024.

Nous voulons traiter les données dans l'ordre chronologique. Nous devons donc pour la première période récupérer les données annuelles et les transformer en trimestrielles, nous choisissons d'interpoler les données manquantes de manière linéaire en donnant la valeur annuelle au Q1. Pour cela nous avons besoin de traiter les données de 1998 à 2007 avant afin de pouvoir faire l'interpolation entre le Q1 de 1997 et celui de 1998.

Nous récupérons donc d'abord toutes les données


In [112]:
# Chargement des données de 1995 à 1997 
df_working_hours_95_97 = pd.read_csv("Données_extraites/Working_hours_eurostat_1995-1997.csv", encoding='utf-8')

# Sélection des colonnes nécessaires avec les pays sélectionnés
df_working_hours_95_97_selected = df_working_hours_95_97[df_working_hours_95_97['geo'].isin(countries_selected)][['geo', 'TIME_PERIOD', 'OBS_VALUE']]

# Modification du format pour avoir les périodes en index et les noms des pays en nom de colonnes
df_working_hours_95_97_pivot = df_working_hours_95_97_selected.pivot_table(index='TIME_PERIOD', columns='geo', values='OBS_VALUE')

# Transformation des colonnes en WH_Nom_du_pays
df_working_hours_95_97_pivot.columns = [f'WH_{col}' for col in df_working_hours_95_97_pivot.columns]
df_working_hours_95_97_pivot.columns = [col.replace(" ", "_") for col in df_working_hours_95_97_pivot.columns]

# Même processus pour celles de 1998 à 2007 et celles de 2008 à 2024
df_working_hours_98_07 = pd.read_csv("Données_extraites/Working_hours_eurostat_1998-2007.csv", encoding='utf-8')
df_working_hours_98_07_selected = df_working_hours_98_07[df_working_hours_98_07['geo'].isin(countries_selected)][['geo', 'TIME_PERIOD', 'OBS_VALUE']]
df_working_hours_98_07_pivot = df_working_hours_98_07_selected.pivot_table(index='TIME_PERIOD', columns='geo', values='OBS_VALUE')
df_working_hours_98_07_pivot.columns = [f'WH_{col}' for col in df_working_hours_98_07_pivot.columns]
df_working_hours_98_07_pivot.columns = [col.replace(" ", "_") for col in df_working_hours_98_07_pivot.columns]

df_working_hours_08_24 = pd.read_csv("Données_extraites/Working_hours_eurostat_2008-2024.csv", encoding='utf-8')
df_working_hours_08_24_selected = df_working_hours_08_24[df_working_hours_08_24['geo'].isin(countries_selected)][['geo', 'TIME_PERIOD', 'OBS_VALUE']]
df_working_hours_08_24_pivot = df_working_hours_08_24_selected.pivot_table(index='TIME_PERIOD', columns='geo', values='OBS_VALUE')
df_working_hours_08_24_pivot.columns = [f'WH_{col}' for col in df_working_hours_08_24_pivot.columns]
df_working_hours_08_24_pivot.columns = [col.replace(" ", "_") for col in df_working_hours_08_24_pivot.columns]



Et maintenant nous configurons l'interpolation. Pour clarifier le point précédent, l'interpolation est linéaire, donc va tracer une droite entre deux périodes avec une valeur présente et inférer une valeur aux périodes pour laquelle la valeur est manquante et qui se trouvent entre nos deux périodes. Comme nous faisons le choix de donner la valeur annuelle au Q1 il faut ajouter une valeur de 1998 pour pouvoir inférer les valeurs du Q2, Q3 et Q4 de 1997, ce que nous faisons donc en ajoutant le Q1 de 1998 à notre table

In [113]:
# Récupération du Q1 de 1998 et suppresion de la table pour éviter les doublons lors de la concaténation
Q1_1998 =  df_working_hours_98_07_pivot.loc[['1998-Q1']]
df_working_hours_98_07_pivot = df_working_hours_98_07_pivot.drop(index='1998-Q1', errors='ignore')

# Ajout des trimestres vierges aux données de 1995 à 1998 pour ensuite les compléter par interpolation
new_index = []
values = []
for year in df_working_hours_95_97_pivot.index:
    # Ajout du Q1 pour chaque année (avec la valeur annuelle)
    new_index.append(f"{year}-Q1")
    values.append(df_working_hours_95_97_pivot.loc[year].values)
    
    # Ajouts des Q2, Q3 et Q4 (lignes vides pour l'instant)
    for quarter in ['Q2', 'Q3', 'Q4']:
        new_index.append(f"{year}-{quarter}")
        values.append([None] * len(df_working_hours_95_97_pivot.columns))

# Création du dataframe avec les lignes vides
df_working_hours_95_97_pivot = pd.DataFrame(values, index=new_index, columns=df_working_hours_95_97_pivot.columns)
df_working_hours_95_97_pivot.index.name = 'TIME_PERIOD'

df_working_hours_95_97_pivot = pd.concat([df_working_hours_95_97_pivot, Q1_1998])

# Interpolation
df_working_hours_95_97_pivot = df_working_hours_95_97_pivot.interpolate(method='linear')

Maintenant nous procédons à la concaténation des données pour avoir un unique dataframe contenant les heures de travail trimestrielles de 1995 à 2024. Seulement nous devons supprimer les index de 2008 dans le dataframe de 1998 à 2007 car sinon ils seraient en double lors de la concaténation. 

In [114]:
# Suppresion des index
df_working_hours_98_07_pivot = df_working_hours_98_07_pivot.drop(index = ['2008-Q1','2008-Q2', '2008-Q3','2008-Q4'] , errors='ignore')

# Concaténation
df_working_hours = pd.concat(
    [df_working_hours_95_97_pivot, df_working_hours_98_07_pivot, df_working_hours_08_24_pivot], 
    axis=0
)

In [115]:
#Nettoyage de données
'''
on applique la même fonction d'interpolation qu'on a utilisée pour df_working_hours_95_97_pivot
#Ceci permet d'eliminer les valeurs manquantes, lorsqu'elles ne sont pas situées au début de la '
période temporelle considérée

Pour les valeurs manquantes restantes (début ou fin du dataframe--> c'est à revoir)
'''

df_working_hours=df_working_hours.interpolate(method='linear')
df_working_hours


,WH_Austria,WH_Belgium,WH_Czechia,WH_Denmark,WH_Estonia,WH_Finland,WH_France,WH_Germany,WH_Greece,WH_Hungary,...,WH_Romania,WH_Slovenia,WH_Spain,WH_Sweden,WH_Switzerland,WH_United_Kingdom,WH_Bulgaria,WH_Latvia,WH_Lithuania,WH_Slovakia
TIME_PERIOD,,,,,,,,,,,,,,,,,,,,,
1995-Q1,40.00,37.8,NaN,34.300,NaN,36.900,39.700,38.7,42.3,NaN,...,NaN,NaN,38.300,34.0,NaN,36.90,NaN,NaN,NaN,NaN
1995-Q2,39.95,37.8,NaN,34.225,NaN,37.075,39.675,38.7,42.2,NaN,...,NaN,NaN,38.225,34.1,NaN,36.85,NaN,NaN,NaN,NaN
1995-Q3,39.90,37.8,NaN,34.150,NaN,37.250,39.650,38.7,42.1,NaN,...,NaN,NaN,38.150,34.2,NaN,36.80,NaN,NaN,NaN,NaN
1995-Q4,39.85,37.8,NaN,34.075,NaN,37.425,39.625,38.7,42.0,NaN,...,NaN,NaN,38.075,34.3,NaN,36.75,NaN,NaN,NaN,NaN
1996-Q1,39.80,37.8,NaN,34.000,NaN,37.600,39.600,38.7,41.9,41.2,...,NaN,42.2,38.000,34.4,37.0,36.70,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-Q4,32.90,34.1,37.8,33.000,36.3,33.600,35.900,33.5,39.5,36.4,...,39.1,37.8,35.400,35.5,36.3,35.70,39.0,38.4,37.6,37.2
2024-Q1,34.00,35.0,37.7,32.600,35.9,34.200,36.200,33.7,38.9,37.0,...,39.2,37.7,36.100,35.3,35.7,35.70,39.3,38.3,37.8,38.0
2024-Q2,32.90,33.8,37.4,32.800,36.4,34.200,34.700,33.5,40.0,37.8,...,38.2,37.5,36.400,33.9,35.9,35.70,38.5,38.1,38.3,37.6


### Données des taux d'intérêts <a class="anchor" id="partie23"></a>

Pour les taux d'intérêts nous récupérons des taux de long-terme, à savoir des obligations d'Etat à 10 ans, et des taux de court-terme. Les taux à court terme sont généralement soit le taux interbancaire à trois mois lié aux prêts accordés et acceptés par les banques pour tout excès ou manque de liquidité sur plusieurs mois, soit le taux associé aux bons du Trésor, aux certificats de dépôt ou à des instruments comparables, chacun d'une durée de trois mois. Pour les pays de la zone euro, le « European Interbank Offered Rate » à trois mois est utilisé à partir de la date à laquelle le pays a rejoint l'euro.  

Nous commençons par récupérer les taux de long-terme puis ceux de court-terme

In [116]:
# Chargement des données
df_LT_IR = pd.read_csv('Données_extraites/Long_term_IR_OCDE.csv', encoding='utf-8')

# Sélection des colonnes nécessaires avec les pays sélectionnés
df_LT_IR_selected = df_LT_IR[df_LT_IR['Reference area'].isin(countries_selected)][['Reference area', 'TIME_PERIOD', 'OBS_VALUE']]

# Modification du format pour avoir les périodes en index et les noms des pays en nom de colonnes
df_LT_IR = df_LT_IR_selected.pivot_table(index='TIME_PERIOD', columns='Reference area', values='OBS_VALUE')

# Modification des colonnes pour inclure le préfixe LT_IR_ (=Long Term Interest Rate)
df_LT_IR.columns = [f'LT_IR_{col}' for col in df_LT_IR.columns]
df_LT_IR.columns = [col.replace(" ", "_") for col in df_LT_IR.columns]
df_LT_IR = df_LT_IR.rename(columns={'LT_IR_Slovak_Republic': 'LT_IR_Slovakia'})

# Même processus pour les taux de long-terme
df_ST_IR=pd.read_csv('Données_extraites/Short_term_IR_OCDE.csv', encoding='utf-8')
df_ST_IR_selected = df_ST_IR[df_ST_IR['Reference area'].isin(countries_selected)][['Reference area', 'TIME_PERIOD', 'OBS_VALUE']]
df_ST_IR = df_ST_IR_selected.pivot_table(index='TIME_PERIOD', columns='Reference area', values='OBS_VALUE')
df_ST_IR.columns = [f'ST_IR_{col}' for col in df_ST_IR.columns]
df_ST_IR.columns = [col.replace(" ", "_") for col in df_ST_IR.columns]
df_ST_IR = df_ST_IR.rename(columns={'ST_IR_Slovak_Republic': 'ST_IR_Slovakia'})

#Nettoyage des données manquantes intermédiaires avec une interpolation
df_ST_IR['ST_IR_Hungary']=df_ST_IR['ST_IR_Hungary'].interpolate(method='linear')
df_ST_IR['ST_IR_Slovakia']=df_ST_IR['ST_IR_Slovakia'].interpolate(method='linear')

### Données d'indice à la consommation des prix <a class="anchor" id="partie24"></a>

Nous récupérons les données relatives à l'indice à la consommation des prix, qui nécessite des modifications car les données actuelles sont mensuelles et non trimestrielles. Nous faisons le choix de les transformer en trimestrielles en attribuant les moyennes mensuelles sur trois mois à notre valeur. 

In [117]:
# Chargement des données
df_CPI=pd.read_csv('Données_extraites/CPI_annual_eurostat_1996-2024.csv', encoding='utf-8')

# Sélection des colonnes nécessaires avec les pays sélectionnés
df_CPI_selected = df_CPI[df_CPI['geo'].isin(countries_selected)][['geo', 'TIME_PERIOD', 'OBS_VALUE']]

# Modification du format pour avoir les périodes en index et les noms des pays en nom de colonnes
df_CPI = df_CPI_selected.pivot_table(index='TIME_PERIOD', columns='geo', values='OBS_VALUE')

# Transformation des colonnes en CPI_Nom_du_pays
df_CPI.columns = [f'CPI_{col}' for col in df_CPI.columns]
df_CPI.columns = [col.replace(" ", "_") for col in df_CPI.columns]

Nous transformons ici les données mensuelles en trimestrielles.

In [118]:
# Conversion des mois en trimestre
df_CPI.index = pd.PeriodIndex(df_CPI.index, freq='Q')

# Pour chaque trimestre on a donc 3 valeurs on fait donc la moyenne
df_CPI = df_CPI.groupby(df_CPI.index).mean()

# Modification du format de l'index qui est YYYYQQ en YYYY-QQ pour l'accorder aux autres tables
df_CPI.index = df_CPI.index.astype(str).str.replace(r'(\d{4})Q(\d)', r'\1-Q\2', regex=True)

### Données de croissance potentielle <a class="anchor" id="partie25"></a>

Nous récupérons les données relatives à l'étude de la croissance potentielle sur un dataframe de la banque mondiale. Il nous faut faire attenion aux noms donnés à la Tchéquie et la Slovaquie qui sont Czech Republic et Slovak Republic et non Czechia et Slovakia comme nous le nommons dans notre nomenclature.

De plus les données sont annuelles, donc pour les rendre trimestrielles nous les divisons par 4. 

In [119]:
# Chargement des données 
df_potential_growth= pd.read_excel("Données_extraites/potential_growth_database.xlsx",sheet_name="PotentialGrowthMeasures")

# Modification des colonnes (pour garder une homogénéité entre les différents df)
df_potential_growth= df_potential_growth.rename(columns={'Country':'geo', 'Year':'TIME_PERIOD','HP: Potential real GDP growth (percent)':'OBS_VALUE'})
df_potential_growth = df_potential_growth[df_potential_growth['TIME_PERIOD'] >= 1995]

# Choix des colonnes nécessaires avec les pays sélectionnés. 
df_potential_growth_selected = df_potential_growth[df_potential_growth['geo'].isin(countries_selected)][['geo', 'TIME_PERIOD', 'OBS_VALUE']]

# Modification du format pour avoir les périodes en index et les noms des pays en nom de colonnes
df_potential_growth= df_potential_growth_selected.pivot_table(index='TIME_PERIOD', columns='geo', values='OBS_VALUE')

# Transformation des colonnes en P_Growth_pays
df_potential_growth = df_potential_growth.rename(columns={'Czech Republic': 'Czechia'})
df_potential_growth = df_potential_growth.rename(columns={'Slovak Republic': 'Slovakia'})
df_potential_growth.columns = [f'P_Growth_{col}' for col in df_potential_growth.columns]
df_potential_growth.columns = [col.replace(" ", "_") for col in df_potential_growth.columns]

On s'occupe maintenant de transformer les index dans un format trimestriel, pour cela on va procéder à une interpolation polynomiale 

In [120]:
# Assurer que l'index est au format datetime
df_potential_growth.index = pd.to_datetime(df_potential_growth.index, format="%Y")

# Convertir les années en format numérique pour l'interpolation
years = df_potential_growth.index.year

# Générer un nouvel index trimestriel (ex: 2000.0, 2000.25, 2000.5, 2000.75, ...)
new_index = np.arange(years.min(), years.max() + 0.75, 0.25)

# DataFrame final avec interpolations
df_interpolated = pd.DataFrame(index=new_index)

# Fonction pour calculer l'AIC
def compute_aic(y_true, y_pred, k):
    n = len(y_true)
    mse = mean_squared_error(y_true, y_pred)
    aic = n * np.log(mse) + 2 * k
    return aic

# Boucle sur chaque colonne du DataFrame
best_degrees = {}  # Dictionnaire pour stocker le meilleur degré pour chaque colonne
for col in df_potential_growth.columns:
    values = df_potential_growth[col].dropna().values  # Supprimer les NaN
    valid_years = df_potential_growth[col].dropna().index.year  # Années valides (sans NaN)
    
    if len(values) < 2:  # Vérifier qu'on a assez de points pour interpoler
        print(f"Pas assez de données pour interpoler {col}")
        df_interpolated[col] = np.nan
        continue
    
    # Trouver le degré optimal du polynôme
    best_degree = None
    best_aic = np.inf
    for deg in range(1, min(4, len(values))):  # Limiter le degré max à éviter les erreurs
        try:
            poly_fit = np.polyfit(valid_years, values, deg=deg)
            poly_model = np.poly1d(poly_fit)
            predictions = poly_model(valid_years)

            if np.isnan(predictions).any():  # Vérifier si le polynôme donne NaN
                continue

            aic = compute_aic(values, predictions, deg + 1)  # k = deg + 1 paramètres du polynôme
            if aic < best_aic:
                best_aic = aic
                best_degree = deg
        except:
            continue  # Si une erreur survient, on passe au degré suivant

    best_degrees[col] = best_degree  # Stocker le meilleur degré trouvé
    
    # Appliquer l'interpolation polynomiale sur l'index trimestriel
    if best_degree is not None:
        best_poly_fit = np.polyfit(valid_years, values, best_degree)
        best_poly_model = np.poly1d(best_poly_fit)
        df_interpolated[col] = best_poly_model(new_index)
    else:
        df_interpolated[col] = np.nan  # Si pas de modèle valide, mettre NaN

# Générer les labels trimestriels "YYYY-Qx"
quarters = ['Q1', 'Q2', 'Q3', 'Q4']
quarter_labels = [f"{(year)}-{q}" for year in years for q in quarters][:len(df_interpolated)]
df_interpolated.index = quarter_labels  # Appliquer les labels trimestriels

#On renomme df_interpolated pour l'intégrer dans le df final
df_potential_growth=df_interpolated


In [121]:
#df_interpolated

In [122]:
df_potential_growth

,P_Growth_Austria,P_Growth_Belgium,P_Growth_Bulgaria,P_Growth_Czechia,P_Growth_Denmark,P_Growth_Estonia,P_Growth_Finland,P_Growth_France,P_Growth_Germany,P_Growth_Greece,...,P_Growth_Norway,P_Growth_Poland,P_Growth_Portugal,P_Growth_Romania,P_Growth_Slovakia,P_Growth_Slovenia,P_Growth_Spain,P_Growth_Sweden,P_Growth_Switzerland,P_Growth_United_Kingdom
1995-Q1,2.537808,2.472040,4.208218,-0.535359,3.499599,5.179260,4.051294,2.277348,1.850978,2.595246,...,4.367114,4.361677,4.104443,-0.878275,2.738296,3.085727,3.253955,2.930006,1.074443,3.535958
1995-Q2,2.549322,2.485104,4.408204,-0.279126,3.410234,5.323708,4.096866,2.300522,1.797714,2.850794,...,4.277852,4.350320,4.017848,-0.584846,2.869240,3.211422,3.335413,2.959760,1.138621,3.484991
1995-Q3,2.559001,2.496387,4.595925,-0.033323,3.322411,5.456657,4.136142,2.321061,1.746953,3.086500,...,4.189850,4.338962,3.930888,-0.302486,2.994803,3.327799,3.409695,2.986914,1.200537,3.434556
1995-Q4,2.566884,2.505928,4.771623,0.202229,3.236131,5.578363,4.169273,2.339022,1.698644,3.302809,...,4.103107,4.327605,3.843612,-0.031012,3.115063,3.435067,3.476958,3.011524,1.260221,3.384654
1996-Q1,2.573010,2.513766,4.935537,0.427708,3.151393,5.689082,4.196409,2.354465,1.652738,3.500166,...,4.017623,4.316247,3.756069,0.229762,3.230096,3.533436,3.537357,3.033645,1.317706,3.335283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-Q3,0.968013,1.235680,2.612956,2.467832,2.328887,4.653425,1.983992,1.141845,0.630756,2.782501,...,1.748647,3.203216,1.833218,3.616306,1.454337,4.054811,1.537043,2.081372,1.449721,1.078950
2020-Q4,0.993584,1.265211,2.797347,2.577939,2.396839,4.944137,2.158783,1.198513,0.572377,3.303977,...,1.787825,3.191859,1.962641,3.728136,1.428673,4.314484,1.694721,2.131189,1.447650,1.082273
2021-Q1,1.021304,1.296885,2.993937,2.695789,2.466334,5.249472,2.342553,1.258533,0.511345,3.850954,...,1.828263,3.180501,2.096714,3.847659,1.405393,4.586314,1.861228,2.184055,1.446574,1.086128
2021-Q2,1.051213,1.330738,3.202964,2.821559,2.537371,5.569687,2.535451,1.321963,0.447608,4.423877,...,1.869960,3.169144,2.235486,3.975058,1.384575,4.870511,2.036719,2.240024,1.446526,1.090515


## Création de la base de données <a class="anchor" id="partie3"></a>

Maintenant que tous les dataframes sont construits pour les variables d'intérêts nous concaténons les dataframes pour en avoir un unique qui servira de base de données pour la suite

In [123]:
# Concaténation des DataFrames
df_concat = pd.concat([df_GDP, df_CPI,df_LT_IR, df_ST_IR, df_working_hours, df_potential_growth], axis=1)

# Réorganisation des colonnes pour que celles de chaque pays soient côte à côte
ordered_columns = []

# Modification la liste de base pour qu'elle corresponde au format de notre dataframe
countries_selected.remove('Slovak Republic')
countries_selected.remove('United Kingdom')
countries_selected.remove('Czech Republic')
countries_selected.append('United_Kingdom')

# Tri
for country in countries_selected:
    ordered_columns.append(f'CPI_{country}')
    ordered_columns.append(f'PIB_{country}')
    ordered_columns.append(f'LT_IR_{country}')
    ordered_columns.append(f'ST_IR_{country}')
    ordered_columns.append(f'WH_{country}')
    ordered_columns.append(f'P_Growth_{country}')

df_final = df_concat[ordered_columns]

# Exportation
df_final.to_excel('base_de_données_v1.xlsx', index=True, engine='openpyxl')
df_final

,CPI_Austria,PIB_Austria,LT_IR_Austria,ST_IR_Austria,WH_Austria,P_Growth_Austria,CPI_Belgium,PIB_Belgium,LT_IR_Belgium,ST_IR_Belgium,...,LT_IR_Switzerland,ST_IR_Switzerland,WH_Switzerland,P_Growth_Switzerland,CPI_United_Kingdom,PIB_United_Kingdom,LT_IR_United_Kingdom,ST_IR_United_Kingdom,WH_United_Kingdom,P_Growth_United_Kingdom
1995-Q1,NaN,43186.6,7.627367,5.043334,40.00,2.537808,NaN,53331.5,8.293333,5.706666,...,5.275333,NaN,NaN,1.074443,NaN,257589.8,8.661467,6.722410,36.90,3.535958
1995-Q2,NaN,46786.8,7.184033,4.680000,39.95,2.549322,NaN,56119.3,7.590000,5.050000,...,4.947333,NaN,NaN,1.138621,NaN,248722.5,8.242567,6.742444,36.85,3.484991
1995-Q3,NaN,46335.5,6.997967,4.373333,39.90,2.559001,NaN,53741.7,7.186666,4.416667,...,4.714667,NaN,NaN,1.200537,NaN,255889.4,8.108933,6.849310,36.80,3.434556
1995-Q4,NaN,47362.4,6.730933,4.176667,39.85,2.566884,NaN,57115.3,6.853333,3.960000,...,4.175667,NaN,NaN,1.260221,NaN,263308.8,7.788167,6.696187,36.75,3.384654
1996-Q1,71.553333,44641.2,6.396633,3.476667,39.80,2.573010,70.003333,54240.0,6.640000,3.350000,...,4.400333,NaN,37.0,1.317706,68.1,262343.7,7.753200,6.277534,36.70,3.335283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-Q4,132.593333,122280.3,3.125533,3.957456,32.90,NaN,127.550000,158085.7,3.150000,3.957456,...,0.841333,1.718106,36.3,NaN,NaN,NaN,4.234600,5.326667,35.70,NaN
2024-Q1,133.326667,117154.2,2.835333,3.923615,34.00,NaN,129.700000,147812.8,2.890000,3.923615,...,0.765667,1.628413,35.7,NaN,NaN,NaN,4.029300,5.203333,35.70,NaN
2024-Q2,134.366667,120629.4,3.013533,3.808172,32.90,NaN,131.086667,154840.7,3.063333,3.808172,...,0.722333,1.475667,35.9,NaN,NaN,NaN,4.201500,5.166667,35.70,NaN
2024-Q3,133.950000,118597.7,2.809900,3.555377,34.50,NaN,131.900000,147978.8,2.893333,3.555377,...,0.437000,1.094031,36.4,NaN,NaN,NaN,3.996100,4.946667,35.70,NaN
